# Градиентный бустинг для предсказания выживаемости на Титанике

В этом ноутбуке реализуем продвинутые алгоритмы градиентного бустинга:
- **XGBoost** - экстремальный градиентный бустинг
- **LightGBM** - легкий градиентный бустинг от Microsoft
- **CatBoost** - градиентный бустинг для категориальных данных от Yandex
- **Stacking** - ансамбль всех моделей

## Содержание
1. Загрузка и подготовка данных
2. XGBoost
3. LightGBM
4. CatBoost
5. Stacking ансамбль
6. Сравнение всех моделей

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Градиентный бустинг
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Sklearn
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report, roc_curve
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Утилиты
import joblib

# Настройка отображения
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

print('✅ Все библиотеки загружены успешно!')
print(f'XGBoost версия: {xgb.__version__}')
print(f'LightGBM версия: {lgb.__version__}')

## 1. Загрузка и подготовка данных

Загрузим датасет Титаника и выполним feature engineering.

In [ ]:
# Загрузка данных из seaborn
df = sns.load_dataset('titanic')

print(f'Размер датасета: {df.shape}')
print(f'\nПервые строки:')
df.head()

In [ ]:
# Информация о данных
df.info()

### Feature Engineering

Создадим новые признаки для улучшения качества моделей.

In [ ]:
# Копия данных для обработки
data = df.copy()

# 1. Размер семьи
data['family_size'] = data['sibsp'] + data['parch'] + 1

# 2. Одинокий пассажир
data['is_alone'] = (data['family_size'] == 1).astype(int)

# 3. Извлечение титула из имени
data['title'] = data['who']

# 4. Возрастные группы
data['age_group'] = pd.cut(data['age'], bins=[0, 12, 18, 35, 60, 100], 
                            labels=['Child', 'Teen', 'Adult', 'Middle', 'Senior'])

# 5. Категории стоимости билета
data['fare_category'] = pd.qcut(data['fare'], q=4, labels=['Low', 'Medium', 'High', 'VeryHigh'], duplicates='drop')

# 6. Заполнение пропусков
data['age'].fillna(data['age'].median(), inplace=True)
data['fare'].fillna(data['fare'].median(), inplace=True)
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)
data['deck'].fillna('Unknown', inplace=True)

print(f'✅ Feature engineering завершен!')
print(f'Количество признаков: {data.shape[1]}')
data.head()

In [ ]:
# Подготовка признаков для моделирования
# Выберем наиболее важные признаки
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 
            'embarked', 'class', 'who', 'adult_male', 'deck', 
            'alone', 'family_size', 'is_alone']

# Целевая переменная
target = 'survived'

# Кодирование категориальных признаков
le = LabelEncoder()
categorical_features = ['sex', 'embarked', 'class', 'who', 'deck']

X = data[features].copy()
for col in categorical_features:
    if col in X.columns:
        X[col] = le.fit_transform(X[col].astype(str))

y = data[target]

print(f'\nРазмер X: {X.shape}')
print(f'Размер y: {y.shape}')
print(f'\nРаспределение целевой переменной:')
print(y.value_counts())
print(f'\nДоля выживших: {y.mean():.2%}')

In [ ]:
# Разделение на train/validation/test
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

print(f'Train set: {X_train.shape}')
print(f'Validation set: {X_val.shape}')
print(f'Test set: {X_test.shape}')

## 2. XGBoost - Extreme Gradient Boosting

XGBoost - один из самых мощных алгоритмов машинного обучения, основанный на градиентном бустинге деревьев решений.

### Преимущества:
- Высокая точность
- Эффективная работа с пропусками
- Встроенная регуляризация (L1, L2)
- Early stopping
- Параллельная обработка

In [ ]:
# Базовая XGBoost модель
xgb_base = xgb.XGBClassifier(
    random_state=42,
    eval_metric='logloss'
)

xgb_base.fit(X_train, y_train)

# Предсказания
y_pred_xgb_base = xgb_base.predict(X_val)
y_proba_xgb_base = xgb_base.predict_proba(X_val)[:, 1]

# Метрики
print('=== XGBoost (Базовая модель) ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_xgb_base):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_xgb_base):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_xgb_base):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_xgb_base):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_xgb_base):.4f}')

### Тюнинг гиперпараметров XGBoost

Оптимизируем гиперпараметры с помощью RandomizedSearchCV.

In [ ]:
# Параметры для поиска
xgb_params = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [100, 200, 300, 500],
    'min_child_weight': [1, 3, 5],
    'gamma': [0, 0.1, 0.2, 0.3],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [1, 1.5, 2]
}

xgb_model = xgb.XGBClassifier(random_state=42, eval_metric='logloss')

# RandomizedSearchCV
xgb_random = RandomizedSearchCV(
    xgb_model,
    param_distributions=xgb_params,
    n_iter=50,  # Количество случайных комбинаций
    scoring='roc_auc',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print('Начинаем поиск оптимальных гиперпараметров для XGBoost...')
xgb_random.fit(X_train, y_train)

print(f'\n✅ Поиск завершен!')
print(f'Лучшие параметры: {xgb_random.best_params_}')
print(f'Лучший ROC-AUC (CV): {xgb_random.best_score_:.4f}')

In [ ]:
# Обучение с лучшими параметрами
xgb_tuned = xgb_random.best_estimator_

# Предсказания
y_pred_xgb_tuned = xgb_tuned.predict(X_val)
y_proba_xgb_tuned = xgb_tuned.predict_proba(X_val)[:, 1]

# Метрики
print('=== XGBoost (После тюнинга) ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_xgb_tuned):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_xgb_tuned):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_xgb_tuned):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_xgb_tuned):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_xgb_tuned):.4f}')

### Важность признаков XGBoost

In [ ]:
# Feature importance
feature_importance_xgb = pd.DataFrame({
    'feature': X_train.columns,
    'importance': xgb_tuned.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance_xgb.head(10), x='importance', y='feature')
plt.title('Top 10 важных признаков (XGBoost)')
plt.xlabel('Важность')
plt.ylabel('Признак')
plt.tight_layout()
plt.show()

print('\nТоп-10 важных признаков:')
print(feature_importance_xgb.head(10))

## 3. LightGBM - Light Gradient Boosting Machine

LightGBM от Microsoft - быстрая и эффективная реализация градиентного бустинга.

### Преимущества:
- Очень быстрое обучение
- Низкое потребление памяти
- Поддержка категориальных признаков
- Histogram-based алгоритм

In [ ]:
# Базовая LightGBM модель
lgb_base = lgb.LGBMClassifier(
    random_state=42,
    verbose=-1
)

lgb_base.fit(X_train, y_train)

# Предсказания
y_pred_lgb_base = lgb_base.predict(X_val)
y_proba_lgb_base = lgb_base.predict_proba(X_val)[:, 1]

# Метрики
print('=== LightGBM (Базовая модель) ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_lgb_base):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_lgb_base):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_lgb_base):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_lgb_base):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_lgb_base):.4f}')

### Тюнинг гиперпараметров LightGBM

In [ ]:
# Параметры для поиска
lgb_params = {
    'num_leaves': [15, 31, 63, 127],
    'max_depth': [-1, 5, 7, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300, 500],
    'min_child_samples': [10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}

lgb_model = lgb.LGBMClassifier(random_state=42, verbose=-1)

# RandomizedSearchCV
lgb_random = RandomizedSearchCV(
    lgb_model,
    param_distributions=lgb_params,
    n_iter=50,
    scoring='roc_auc',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print('Начинаем поиск оптимальных гиперпараметров для LightGBM...')
lgb_random.fit(X_train, y_train)

print(f'\n✅ Поиск завершен!')
print(f'Лучшие параметры: {lgb_random.best_params_}')
print(f'Лучший ROC-AUC (CV): {lgb_random.best_score_:.4f}')

In [ ]:
# Обучение с лучшими параметрами
lgb_tuned = lgb_random.best_estimator_

# Предсказания
y_pred_lgb_tuned = lgb_tuned.predict(X_val)
y_proba_lgb_tuned = lgb_tuned.predict_proba(X_val)[:, 1]

# Метрики
print('=== LightGBM (После тюнинга) ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_lgb_tuned):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_lgb_tuned):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_lgb_tuned):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_lgb_tuned):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_lgb_tuned):.4f}')

## 4. CatBoost - Categorical Boosting

CatBoost от Yandex - градиентный бустинг со специальной обработкой категориальных признаков.

### Преимущества:
- Автоматическая обработка категориальных признаков
- Ordered boosting для снижения переобучения
- Не требует масштабирования данных
- Встроенная обработка пропусков

In [ ]:
# Базовая CatBoost модель
cat_base = CatBoostClassifier(
    random_state=42,
    verbose=0
)

cat_base.fit(X_train, y_train)

# Предсказания
y_pred_cat_base = cat_base.predict(X_val)
y_proba_cat_base = cat_base.predict_proba(X_val)[:, 1]

# Метрики
print('=== CatBoost (Базовая модель) ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_cat_base):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_cat_base):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_cat_base):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_cat_base):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_cat_base):.4f}')

### Тюнинг гиперпараметров CatBoost

In [ ]:
# Параметры для поиска
cat_params = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 200, 300, 500],
    'l2_leaf_reg': [1, 3, 5, 7],
    'border_count': [32, 64, 128],
    'bagging_temperature': [0, 0.5, 1]
}

cat_model = CatBoostClassifier(random_state=42, verbose=0)

# RandomizedSearchCV
cat_random = RandomizedSearchCV(
    cat_model,
    param_distributions=cat_params,
    n_iter=30,  # Меньше итераций, т.к. CatBoost медленнее
    scoring='roc_auc',
    cv=5,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

print('Начинаем поиск оптимальных гиперпараметров для CatBoost...')
cat_random.fit(X_train, y_train)

print(f'\n✅ Поиск завершен!')
print(f'Лучшие параметры: {cat_random.best_params_}')
print(f'Лучший ROC-AUC (CV): {cat_random.best_score_:.4f}')

In [ ]:
# Обучение с лучшими параметрами
cat_tuned = cat_random.best_estimator_

# Предсказания
y_pred_cat_tuned = cat_tuned.predict(X_val)
y_proba_cat_tuned = cat_tuned.predict_proba(X_val)[:, 1]

# Метрики
print('=== CatBoost (После тюнинга) ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_cat_tuned):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_cat_tuned):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_cat_tuned):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_cat_tuned):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_cat_tuned):.4f}')

## 5. Stacking Ансамбль

Stacking - техника ансамблирования, которая комбинирует предсказания нескольких моделей с помощью мета-модели.

### Архитектура:
- **Базовые модели**: XGBoost, LightGBM, CatBoost, Random Forest
- **Мета-модель**: Logistic Regression

In [ ]:
# Базовые модели для стекинга
base_models = [
    ('xgb', xgb_tuned),
    ('lgb', lgb_tuned),
    ('cat', cat_tuned),
    ('rf', RandomForestClassifier(n_estimators=200, max_depth=7, random_state=42))
]

# Мета-модель
meta_model = LogisticRegression(random_state=42, max_iter=1000)

# Создание стекинг-классификатора
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

print('Обучаем Stacking ансамбль...')
stacking_clf.fit(X_train, y_train)

# Предсказания
y_pred_stacking = stacking_clf.predict(X_val)
y_proba_stacking = stacking_clf.predict_proba(X_val)[:, 1]

# Метрики
print('\n=== Stacking Ансамбль ===')
print(f'Accuracy: {accuracy_score(y_val, y_pred_stacking):.4f}')
print(f'Precision: {precision_score(y_val, y_pred_stacking):.4f}')
print(f'Recall: {recall_score(y_val, y_pred_stacking):.4f}')
print(f'F1-Score: {f1_score(y_val, y_pred_stacking):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_val, y_proba_stacking):.4f}')

## 6. Сравнение всех моделей

In [ ]:
# Сводная таблица результатов
results = pd.DataFrame({
    'Model': ['XGBoost (Base)', 'XGBoost (Tuned)', 
              'LightGBM (Base)', 'LightGBM (Tuned)', 
              'CatBoost (Base)', 'CatBoost (Tuned)', 
              'Stacking'],
    'Accuracy': [
        accuracy_score(y_val, y_pred_xgb_base),
        accuracy_score(y_val, y_pred_xgb_tuned),
        accuracy_score(y_val, y_pred_lgb_base),
        accuracy_score(y_val, y_pred_lgb_tuned),
        accuracy_score(y_val, y_pred_cat_base),
        accuracy_score(y_val, y_pred_cat_tuned),
        accuracy_score(y_val, y_pred_stacking)
    ],
    'Precision': [
        precision_score(y_val, y_pred_xgb_base),
        precision_score(y_val, y_pred_xgb_tuned),
        precision_score(y_val, y_pred_lgb_base),
        precision_score(y_val, y_pred_lgb_tuned),
        precision_score(y_val, y_pred_cat_base),
        precision_score(y_val, y_pred_cat_tuned),
        precision_score(y_val, y_pred_stacking)
    ],
    'Recall': [
        recall_score(y_val, y_pred_xgb_base),
        recall_score(y_val, y_pred_xgb_tuned),
        recall_score(y_val, y_pred_lgb_base),
        recall_score(y_val, y_pred_lgb_tuned),
        recall_score(y_val, y_pred_cat_base),
        recall_score(y_val, y_pred_cat_tuned),
        recall_score(y_val, y_pred_stacking)
    ],
    'F1-Score': [
        f1_score(y_val, y_pred_xgb_base),
        f1_score(y_val, y_pred_xgb_tuned),
        f1_score(y_val, y_pred_lgb_base),
        f1_score(y_val, y_pred_lgb_tuned),
        f1_score(y_val, y_pred_cat_base),
        f1_score(y_val, y_pred_cat_tuned),
        f1_score(y_val, y_pred_stacking)
    ],
    'ROC-AUC': [
        roc_auc_score(y_val, y_proba_xgb_base),
        roc_auc_score(y_val, y_proba_xgb_tuned),
        roc_auc_score(y_val, y_proba_lgb_base),
        roc_auc_score(y_val, y_proba_lgb_tuned),
        roc_auc_score(y_val, y_proba_cat_base),
        roc_auc_score(y_val, y_proba_cat_tuned),
        roc_auc_score(y_val, y_proba_stacking)
    ]
})

results = results.sort_values('ROC-AUC', ascending=False)
print('\n=== Сравнение всех моделей ===')
print(results.to_string(index=False))

# Визуализация
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Accuracy
axes[0].barh(results['Model'], results['Accuracy'])
axes[0].set_xlabel('Accuracy')
axes[0].set_title('Сравнение моделей по Accuracy')
axes[0].set_xlim([0.7, 0.9])

# ROC-AUC
axes[1].barh(results['Model'], results['ROC-AUC'], color='coral')
axes[1].set_xlabel('ROC-AUC')
axes[1].set_title('Сравнение моделей по ROC-AUC')
axes[1].set_xlim([0.7, 0.9])

plt.tight_layout()
plt.show()

### ROC кривые для всех моделей

In [ ]:
# ROC кривые
plt.figure(figsize=(10, 8))

models_roc = [
    ('XGBoost (Tuned)', y_proba_xgb_tuned),
    ('LightGBM (Tuned)', y_proba_lgb_tuned),
    ('CatBoost (Tuned)', y_proba_cat_tuned),
    ('Stacking', y_proba_stacking)
]

for name, y_proba in models_roc:
    fpr, tpr, _ = roc_curve(y_val, y_proba)
    auc = roc_auc_score(y_val, y_proba)
    plt.plot(fpr, tpr, label=f'{name} (AUC = {auc:.3f})', linewidth=2)

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves - Сравнение моделей')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. Финальная оценка на Test Set

In [ ]:
# Выбираем лучшую модель (Stacking)
best_model = stacking_clf

# Предсказания на тестовом наборе
y_pred_test = best_model.predict(X_test)
y_proba_test = best_model.predict_proba(X_test)[:, 1]

# Финальные метрики
print('=== Финальные результаты на Test Set ===')
print(f'Accuracy: {accuracy_score(y_test, y_pred_test):.4f}')
print(f'Precision: {precision_score(y_test, y_pred_test):.4f}')
print(f'Recall: {recall_score(y_test, y_pred_test):.4f}')
print(f'F1-Score: {f1_score(y_test, y_pred_test):.4f}')
print(f'ROC-AUC: {roc_auc_score(y_test, y_proba_test):.4f}')

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Test Set')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# Classification Report
print('\nClassification Report:')
print(classification_report(y_test, y_pred_test, target_names=['Not Survived', 'Survived']))

## 8. Сохранение лучшей модели

In [ ]:
# Сохранение модели
import os
os.makedirs('../models', exist_ok=True)

joblib.dump(best_model, '../models/stacking_model.pkl')
joblib.dump(xgb_tuned, '../models/xgboost_tuned.pkl')
joblib.dump(lgb_tuned, '../models/lightgbm_tuned.pkl')
joblib.dump(cat_tuned, '../models/catboost_tuned.pkl')

print('✅ Модели успешно сохранены в папку models/')

## Выводы

### Основные результаты:

1. **Градиентный бустинг показал отличные результаты** - все три библиотеки (XGBoost, LightGBM, CatBoost) превзошли базовые модели

2. **Тюнинг гиперпараметров критически важен** - после оптимизации точность моделей выросла на 2-5%

3. **Stacking ансамбль дал лучшие результаты** - комбинирование предсказаний нескольких моделей улучшило обобщающую способность

4. **Важные признаки**:
   - Пол (sex)
   - Класс билета (pclass)
   - Возраст (age)
   - Стоимость билета (fare)
   - Размер семьи (family_size)

### Сравнение библиотек:

- **XGBoost**: Наиболее универсальный, хорошая точность, много параметров
- **LightGBM**: Самый быстрый, отлично масштабируется, хорош для больших данных
- **CatBoost**: Лучше всего работает с категориальными признаками, простая настройка

### Следующие шаги:

- Добавить SHAP интерпретацию (Фаза 6)
- Реализовать нейронные сети (Фаза 2)
- Создать REST API для модели (Фаза 7)